In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import torch
device = torch.device("cpu")
print("Using device:", device)


Using device: cpu


In [3]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_49.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_67_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_90.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_77.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_20_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_54.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_106_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_42.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_81.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_107.wav
/kaggle/input/shl-in

In [5]:
BASE_PATH = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset"

AUDIO_TRAIN_PATH = f"{BASE_PATH}/audios/train"
AUDIO_TEST_PATH = f"{BASE_PATH}/audios/test"
CSV_PATH = f"{BASE_PATH}/csvs"

print("Train audio:", os.listdir(AUDIO_TRAIN_PATH)[:3])
print("CSV files:", os.listdir(CSV_PATH))


Train audio: ['audio_49.wav', 'audio_77_2.wav', 'audio_90.wav']
CSV files: ['train.csv', 'test.csv']


In [6]:
import pandas as pd

train_df = pd.read_csv(f"{CSV_PATH}/train.csv")
test_df  = pd.read_csv(f"{CSV_PATH}/test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()


Train shape: (409, 2)
Test shape: (197, 1)


,filename,label
0,audio_173,3.0
1,audio_138,3.0
2,audio_127,2.0
3,audio_95,2.0
4,audio_73,3.5


In [7]:
import torch
import librosa
import numpy as np
from tqdm import tqdm

from transformers import Wav2Vec2Processor, Wav2Vec2Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


E0000 00:00:1766086684.171011      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766086684.291122      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766086685.333408      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766086685.333469      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766086685.333472      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766086685.333475      55 computation_placer.cc:177] computation placer already registered. Please check linka

In [8]:
print("Torch CUDA available:", torch.cuda.is_available())
print("Model loaded successfully")

Torch CUDA available: False
Model loaded successfully


In [9]:
import torch
import librosa
import numpy as np

def extract_embedding(audio_path, processor, model, target_sr=16000):
    speech, sr = librosa.load(audio_path, sr=target_sr)

    inputs = processor(
        speech,
        sampling_rate=target_sr,
        return_tensors="pt",
        padding=True
    )

    with torch.no_grad():
        outputs = model(**inputs)

    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding


In [16]:
import os

print(os.listdir("/kaggle/input/modelhug"))


['wav2vec2_processor', 'wav2vec2_model']


In [27]:
processor_path = "/kaggle/input/modelhug/wav2vec2_processor/content/wav2vec2_processor"
model_path = "/kaggle/input/modelhug/wav2vec2_model/content/wav2vec2_model"

In [28]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model

processor = Wav2Vec2Processor.from_pretrained(processor_path)
model = Wav2Vec2Model.from_pretrained(model_path)

model.eval()
print("Processor and model loaded successfully")


Processor and model loaded successfully


In [29]:
from tqdm import tqdm

X, y = [], []

for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    audio_file = f"{AUDIO_TRAIN_PATH}/{row['filename']}.wav"
    emb = extract_embedding(audio_file, processor, model)
    X.append(emb)
    y.append(row["label"])

X = np.array(X)
y = np.array(y)

print("Feature shape:", X.shape)


  0%|          | 0/409 [00:00<?, ?it/s]/tmp/ipykernel_55/1933669748.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(audio_path, sr=target_sr)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  0%|          | 1/409 [00:47<5:23:25, 47.56s/it]/tmp/ipykernel_55/1933669748.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(audio_path, sr=target_sr)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  1%|          | 3/409 [01:29<3:00:17, 26.64s/it]/tmp/ipykernel_55/1933669748.

Feature shape: (409, 768)


In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_rf = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

model_rf.fit(X_train, y_train)

preds = model_rf.predict(X_val)
mae = mean_absolute_error(y_val, preds)

print("Validation MAE:", mae)


Validation MAE: 0.531280487804878


In [31]:
# Prepare test data
X_test = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    audio_file = f"{AUDIO_TEST_PATH}/{row['filename']}.wav"
    emb = extract_embedding(audio_file, processor, model)
    X_test.append(emb)

X_test = np.array(X_test)
print("Test feature shape:", X_test.shape)

# Predict using trained Random Forest
test_preds = model_rf.predict(X_test)
print("Predictions:", test_preds[:5])

# Save predictions to CSV
test_df['label'] = test_preds
test_df.to_csv("/kaggle/working/test_predictions.csv", index=False)
print("Predictions saved to test_predictions.csv")


  3%|▎         | 6/197 [01:42<1:01:00, 19.16s/it]/tmp/ipykernel_55/1933669748.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(audio_path, sr=target_sr)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  7%|▋         | 13/197 [03:03<35:32, 11.59s/it] /tmp/ipykernel_55/1933669748.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(audio_path, sr=target_sr)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 17%|█▋        | 33/197 [06:27<29:42, 10.87s/it]/tmp/ipykernel_55/19

Test feature shape: (197, 768)
Predictions: [2.6525 3.145  3.1425 3.8525 2.81  ]
Predictions saved to test_predictions.csv


In [33]:
submission = test_df[['filename', 'label']]
submission.to_csv("/kaggle/working/submission.csv", index=False)

In [34]:
!ls -lh /kaggle/working/

total 8.0K
-rw-r--r-- 1 root root 3.1K Dec 18 22:58 submission.csv
-rw-r--r-- 1 root root 3.1K Dec 18 22:29 test_predictions.csv
